In [1]:
import json

import pandas as pd
import pickle
from s3fs.core import S3FileSystem

In [2]:
MODEL_DIR = "mids-capstone-irrigation-detection/models"

In [3]:
s3_file = S3FileSystem()

In [43]:
def f_scores(scores):
    precision = scores[6]
    recall = scores[7]
    if precision + recall == 0.0:
        return [0.0, 0.0]
    f1 = (2 * precision * recall) / (precision + recall)
    beta = 0.5
    f05 = ((1 + beta) * precision * recall) / (beta * precision + recall)
    return [f1, f05]
    

In [44]:
def load_results():
    for file in s3_file.listdir(MODEL_DIR):
        if file['type'] != 'directory':
            continue
        model_type = file["name"].split("/")[-1]
        data = []
        for result in s3_file.glob(file["name"]+"/*.json"):
            model_type = result.split("/")[-2]
            percent = result.split("/")[-1].split("_")[0]
            r = json.load(s3_file.open(result))
            score = [model_type, percent, r["architecture"]] + r["score"] + f_scores(r["score"])
            data.append(score)
        df = pd.DataFrame(data)
        df.columns = ["model_type","split_percent","architecture","loss","tp","fp","tn","fn","accuracy","precision","recall","auc","f1","f0.5"]
    return df


In [45]:
df = load_results()

In [46]:
df

,model_type,split_percent,architecture,loss,tp,fp,tn,fn,accuracy,precision,recall,auc,f1,f0.5
0,supervised_baseline,10,InceptionV3,0.636906,187.0,2821.0,5005.0,19.0,0.646414,0.062168,0.907767,0.865759,0.116366,0.090164
1,supervised_baseline,10,ResNet101V2,0.656869,180.0,2599.0,5227.0,26.0,0.673182,0.064772,0.873786,0.858051,0.120603,0.093685
2,supervised_baseline,10,ResNet50,0.929635,195.0,2906.0,4919.0,12.0,0.636703,0.062883,0.942029,0.867712,0.117896,0.091278
3,supervised_baseline,10,Xception,0.573596,193.0,2905.0,4921.0,13.0,0.636703,0.062298,0.936893,0.876145,0.116828,0.090440
4,supervised_baseline,1,InceptionV3,0.327954,168.0,1805.0,6021.0,38.0,0.770543,0.085150,0.815534,0.864904,0.154199,0.121387
5,supervised_baseline,1,ResNet101V2,0.669254,0.0,0.0,7825.0,207.0,0.974228,0.000000,0.000000,0.500000,0.000000,0.000000
6,supervised_baseline,1,ResNet50,0.193719,0.0,0.0,7826.0,206.0,0.974353,0.000000,0.000000,0.844269,0.000000,0.000000
7,supervised_baseline,1,Xception,0.449320,184.0,2485.0,5340.0,23.0,0.687749,0.068940,0.888889,0.867881,0.127955,0.099549
8,supervised_baseline,25,InceptionV3,0.874272,206.0,7826.0,0.0,0.0,0.025647,0.025647,1.000000,0.591631,0.050012,0.037984
9,supervised_baseline,25,ResNet101V2,0.860707,202.0,3464.0,4361.0,5.0,0.568103,0.055101,0.975845,0.885883,0.104312,0.080382
